In [16]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from skimage import io
from tqdm import tqdm
import os

In [2]:
path = 'C:/Users/admicf/Documents/AppliedDeepLearningClass-master/exercises/ProyectoFinal/'

In [3]:
dataTraining = pd.read_csv(os.path.join(path, 'dataTraining.csv'), encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv(os.path.join(path, 'dataTesting.csv'), encoding='UTF-8', index_col=0)

In [17]:
images_training = []
for i in tqdm(dataTraining.index):
    images_training.append(io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())
images_training = np.stack(images_training)

100%|████████████████████████████████████████████████████| 7895/7895 [00:09<00:00, 819.81it/s]


In [18]:
images_testing = []
for i in tqdm(dataTesting.index):
    images_testing.append(io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())
images_testing = np.stack(images_testing)

100%|████████████████████████████████████████████████████| 3383/3383 [00:03<00:00, 857.08it/s]


In [4]:
vect = CountVectorizer(ngram_range=(1, 2), max_features=1000)
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 1000)

In [5]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [7]:
X_train_text, X_test_text, y_train_genres_text, y_test_genres_text = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=666)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=24, random_state=666))

In [10]:
clf.fit(X_train_text, y_train_genres_text)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=24, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=666, verbose=0, warm_start=False),
          n_jobs=1)

In [ ]:
X_test_dtm = vect.transform(dataTesting['plot'])
y_pred_test_genres_clf = clf.predict_proba(X_test_dtm)

In [ ]:
from sklearn.decomposition import RandomizedPCA
from sklearn.decomposition import PCA
pca = PCA(n_components = 32)
images_training_pca = pca.fit_transform(images_training)
images_testing_pca = pca.transform(images_testing)

In [25]:
X_train_img, X_test_img, y_train_genres_img, y_test_genres_img = train_test_split(images_training_pca, y_genres, test_size=0.20, random_state=666)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
ovr = OneVsRestClassifier(LogisticRegression())
ovr_mod = ovr.fit(X_train_img,y_train_genres_img)

In [31]:
y_pred_test_genres_ovr = ovr_mod.predict_proba(images_testing_pca)

In [33]:
finalpred=(y_pred_test_genres_clf+y_pred_test_genres_ovr)/2
finalpred.shape

(3383, 24)

In [35]:
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']


pd.DataFrame(finalpred, index=dataTesting.index, columns=cols).to_csv('finalpred_ML.csv', index_label='ID')

In [38]:
from keras.models import load_model
model_rn = load_model('model_rn3_2.h5')

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
images_testing_2 = images_testing.reshape(3383, 256, 160,1)
y_pred_test_genres_rn = model_rn.predict_proba(images_testing_2)

In [ ]:
model = load_model('model_text_nn.h5')
X_test_dtm = vect.transform(dataTesting['plot'])
y_pred_test_genres_rn2 = model.predict_proba(X_test_dtm)

In [ ]:
finalpred_NN=(y_pred_test_genres_rn+y_pred_test_genres_rn2)/2
finalpred_NN.shape

In [ ]:
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']


pd.DataFrame(finalpred_NN, index=dataTesting.index, columns=cols).to_csv('finalpred_NN.csv', index_label='ID')